# Feature Enginering

In [1]:
# Импортируем библиотеки для работы с данными
import numpy as np
import pandas as pd

## Импортируем данные

In [2]:
train_df = pd.read_parquet("train_data.pqt")   # Импортируем train и test
test_df = pd.read_parquet("test_data.pqt")

In [3]:
# Посмитрим на входные данные
train_df.head(5)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}


In [4]:
test_df.head(5)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,None,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,None
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}


## Реализуем функции для обработки наших данных
----------------------------------------------
### Цели:
1) Восстановить строки с 'month_4'
2) Заполнить все пропуски 0 и '-'
3) Объеденить все данные по 'id' в 1 строку


In [5]:
# Функция замены пропущенных значений 0 и '-'
def NotNull(df):
    # List_Type_num = ['int64','float64']
    for i in list(df.columns):                 # Проходимся циклом по колонкам DataFrame-а 
        if df[i].dtypes =='float64':           
            df[i] = df[i].fillna(0.)           # Определяем тип столбца и заменяем на соответствующие значения
        if df[i].dtypes == 'object':
            df[i] = df[i].fillna('-')
    return df

In [6]:
#Функция обьединения данных по id (обьединяем данные за 3 месяца в строку)
def inline(df):
    group = df.groupby('id')                     # группируем данные по id
    count_columns = len(df.columns)              # записываем количество столбцов
    def combine_rows(x):
        return pd.Series(x.values.ravel())
    combined = group.apply(combine_rows).reset_index(drop = True)     # обьеденяем данные по id из 3 строк в 1       
#               БЛОК ДЛЯ ОБЬЯДИНЕНИЯ СТРОК С 1 ПОЛЬЗОВАТЕЛЕМ В 1 СТРОКУ с 279 СТОЛБЦАМИ
    spis = {}                 #Создаем словарь {индекс_столбца(int): имя столбца}   
    for n in range(3):                       # Проходимся 3 раза по столбцам исходного df 
        m = '_'+str(n)                       # Добавляем приставку к каждому столбцу для его уникальности
        for i in range(count_columns):
            spis[n*count_columns+i] = df.columns[i] + m
            
    combined.rename(columns = spis,inplace = True)         #переименовываем столбцы с помощью созданного ранее словаря
    combined.drop(['id_1','id_2','date_0','date_1','date_2'], axis= 1 , inplace= True ) # Удаляем ненужные повторяющиеся столбцы
    return combined

In [7]:
# Функция для генерации 4 месяца
def Gen_Month_4(df):                             # На вход подается DataFrame 
    counts = df['id'].value_counts()             # Записываем в переменную количество вхождений каждого id 
    ids =  counts[counts<3].index.tolist()       # Выбирам все id у которых меньше 3 вхождений  Тк у нас отсутствует только month_4 -
                                                    # мы можем утверждать что все id у которых меньше 3-х вхождений не имеют строки со значением month_4
    new_rows = []                                # Создаем список в котором будем хранить созданные строки с month_4
    #МОДУЛЬ ДЛЯ ОБЬЕДИНЕНИЯ ДАННЫХ ПО id
    for uid in ids:                              # Проходимся по всем id у которых меньше 3-х вхождений

        new_row = {'id': uid, 'date': 'month_4'}                    # Создаем строку с mount_4 для id 
        new_row.update({col: None for col in df.columns if col not in ['id', 'date']}) # Заполняем все столбцы кроме id,date значением None   
        new_rows.append(new_row)                               # Добавляем созданную строку в список
    new_rows_df = pd.DataFrame(new_rows)                       # Преобразуем наш список в  DataFrame
    df = pd.concat([df, new_rows_df], ignore_index=True)       # Обьединяем исходный df и созданный нами df
    
    return df.sort_values(by=['id','date']).reset_index(drop=True) # Возвращяем отсортированный df по id и date где у каждого id 3 строки (вхождения) 

## Применим функции для восстановления и преобразования данных

In [8]:
train_df = inline(NotNull(train_df))                 # Заполняем все пропуски и обьединяем строки по  id
test_df = inline(NotNull(Gen_Month_4(test_df)))      # Создаем строки для id без значения month_4, заполняем пропущенные значения, обьединяем строки по id

C:\Users\Grizz\AppData\Local\Temp\ipykernel_11308\2605298474.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_rows_df], ignore_index=True)       # Обьединяем исходный df и созданный нами df


### Сохраним полученные значения

In [9]:
train_df.to_parquet('train_df_postprocess.pqt')
test_df.to_parquet('test_df_postprocess.pqt')

# !!!! ВАЖНО !!!!

Мы совместили все данные по 1 за 3 месяца в 1 строко  
Теперь столбцы с конечным окончанием:  
_0 - данные за 1 месяц  
_1 - данные за 2 месяц  
_2 - данные за 3 месяц  